In [ ]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
import numpy as np
import pickle
import os
from tqdm import tqdm

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load meta
with open(r"D:\food_recommender\embeddings\meta_clean_original_paths.pkl", "rb") as f:
    meta = pickle.load(f)

# Only embed first 3000 images
image_paths = meta["image_path"][:500]

image_embeds = []

for img_path in tqdm(image_paths, desc="Embedding images"):
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        emb = model.get_image_features(**inputs)

    emb = emb / emb.norm(dim=-1, keepdim=True)
    image_embeds.append(emb.cpu().numpy())

image_embeds = np.vstack(image_embeds).astype("float32")
np.save("embeddings/image_embeds_500.npy", image_embeds)

print("DONE:", image_embeds.shape)

In [1]:
import numpy as np
import faiss
import pickle

# ============================
#  Load embeddings + meta
# ============================
EMB_PATH = r"D:\food_recommender\embeddings\image_embeds_500.npy"

# ============================
#  Create FAISS index
# ============================
embeds = np.load(EMB_PATH).astype("float32")
d = embeds.shape[1]   # embedding dimension
index = faiss.IndexFlatL2(d)  # L2 cosine-normalized = works well

index.add(embeds)  # add all embeddings
print("FAISS index size:", index.ntotal)

# ============================
#  Save index
# ============================
faiss.write_index(index, r"D:/food_recommender/embeddings/faiss_image_index_cleaned.bin")
print("Saved FAISS index → faiss_image.index")


FAISS index size: 500
Saved FAISS index → faiss_image.index
